# Spectrogram Visualization
### Code from
https://gist.github.com/parulnith/7f8c174e6ac099e86f0495d3d9a4c01e#file-music_genre_classification-ipynb

In [2]:
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import pickle

#Keras
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import pickle

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

import warnings

1 Physical GPUs, 1 Logical GPUs


## Extract Spectogram Features

In [3]:
filename = '../dataset/gitzan/features_30_sec.csv'
data = pd.read_csv(filename)

data.head()


filename  length  chroma_stft_mean  chroma_stft_var  rms_mean  \
0  blues.00000.wav  661794          0.350088         0.088757  0.130228   
1  blues.00001.wav  661794          0.340914         0.094980  0.095948   
2  blues.00002.wav  661794          0.363637         0.085275  0.175570   
3  blues.00003.wav  661794          0.404785         0.093999  0.141093   
4  blues.00004.wav  661794          0.308526         0.087841  0.091529   

    rms_var  spectral_centroid_mean  spectral_centroid_var  \
0  0.002827             1784.165850          129774.064525   
1  0.002373             1530.176679          375850.073649   
2  0.002746             1552.811865          156467.643368   
3  0.006346             1070.106615          184355.942417   
4  0.002303             1835.004266          343399.939274   

   spectral_bandwidth_mean  spectral_bandwidth_var  ...  mfcc16_var  \
0              2002.449060            85882.761315  ...   52.420910   
1              2039.036516           213843.755497  ...   55.356403   
2              1747.702312            76254.192257  ...   40.598766   
3              1596.412872           166441.494769  ...   44.427753   
4              1748.172116            88445.209036  ...   86.099236   

   mfcc17_mean  mfcc17_var  mfcc18_mean  mfcc18_var  mfcc19_mean  mfcc19_var  \
0    -1.690215   36.524071    -0.408979   41.597103    -2.303523   55.062923   
1    -0.731125   60.314529     0.295073   48.120598    -0.283518   51.106190   
2    -7.729093   47.639427    -1.816407   52.382141    -3.439720   46.639660   
3    -3.319597   50.206673     0.636965   37.319130    -0.619121   37.259739   
4    -5.454034   75.269707    -0.916874   53.613918    -4.404827   62.910812   

   mfcc20_mean  mfcc20_var  label  
0     1.221291   46.936035  blues  
1     0.531217   45.786282  blues  
2    -2.231258   30.573025  blues  
3    -3.407448   31.949339  blues  
4   -11.703234   55.195160  blues  

[5 rows x 60 columns]

In [4]:
data = data.drop(['filename'], axis=1)
print(data.columns)

Index(['length', 'chroma_stft_mean', 'chroma_stft_var', 'rms_mean', 'rms_var',
       'spectral_centroid_mean', 'spectral_centroid_var',
       'spectral_bandwidth_mean', 'spectral_bandwidth_var', 'rolloff_mean',
       'rolloff_var', 'zero_crossing_rate_mean', 'zero_crossing_rate_var',
       'harmony_mean', 'harmony_var', 'perceptr_mean', 'perceptr_var', 'tempo',
       'mfcc1_mean', 'mfcc1_var', 'mfcc2_mean', 'mfcc2_var', 'mfcc3_mean',
       'mfcc3_var', 'mfcc4_mean', 'mfcc4_var', 'mfcc5_mean', 'mfcc5_var',
       'mfcc6_mean', 'mfcc6_var', 'mfcc7_mean', 'mfcc7_var', 'mfcc8_mean',
       'mfcc8_var', 'mfcc9_mean', 'mfcc9_var', 'mfcc10_mean', 'mfcc10_var',
       'mfcc11_mean', 'mfcc11_var', 'mfcc12_mean', 'mfcc12_var', 'mfcc13_mean',
       'mfcc13_var', 'mfcc14_mean', 'mfcc14_var', 'mfcc15_mean', 'mfcc15_var',
       'mfcc16_mean', 'mfcc16_var', 'mfcc17_mean', 'mfcc17_var', 'mfcc18_mean',
       'mfcc18_var', 'mfcc19_mean', 'mfcc19_var', 'mfcc20_mean', 'mfcc20_var',
       'label'

In [5]:
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))
pickle.dump(scaler, open('scaler.pkl','wb'))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [6]:
print(len(X_train))
print(len(X_train[0]))


900
58


In [7]:
len(y_test)

100

In [8]:
X_train[10]

array([-0.29545278,  0.56095006,  1.21097165,  2.047534  ,  1.52299822,
        2.08651298,  0.75501152,  1.90726843, -0.15223599,  2.09421644,
       -0.20513965,  1.51025877,  2.28313929,  0.22217081,  1.54576929,
        0.52710423,  3.10453041, -0.07262036,  1.35819544, -0.25223229,
       -1.83645429, -0.55418618,  0.79232036,  0.45767247, -1.09991947,
        0.94323256,  1.24380472,  0.4882827 , -0.61501917,  1.45250591,
        1.09203412,  0.83363078, -1.14898892,  0.40545525,  1.40149663,
        0.30363115, -0.71169371, -0.34630082,  0.72826424, -0.25948785,
       -0.76323774, -0.12435986,  0.38288141, -0.44220979, -0.65111915,
       -0.72015751,  0.02194085, -0.20362835, -0.61294405, -0.16471299,
        0.35150808,  0.37978645, -0.11651446,  0.52536022,  0.36040044,
        0.62301355, -0.39112107, -0.09386842])

In [9]:
model = Sequential()
model.add(Dense(256, activation='relu', input_shape=(X_train[1].shape[0],)))

model.add(Dense(128, activation='relu'))

model.add(Dense(64, activation='relu'))

model.add(Dense(10, activation='softmax'))


model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train,
                    y_train,
                    epochs=20, batch_size=1)

Epoch 1/20
900/900 [==============================] - 5s 4ms/step - loss: 1.7144 - accuracy: 0.3756
Epoch 2/20
900/900 [==============================] - 4s 4ms/step - loss: 0.9367 - accuracy: 0.6412
Epoch 3/20
900/900 [==============================] - 4s 4ms/step - loss: 0.6042 - accuracy: 0.7845
Epoch 4/20
900/900 [==============================] - 4s 4ms/step - loss: 0.5025 - accuracy: 0.8329
Epoch 5/20
900/900 [==============================] - 4s 4ms/step - loss: 0.4017 - accuracy: 0.8672
Epoch 6/20
900/900 [==============================] - 4s 4ms/step - loss: 0.3124 - accuracy: 0.8997
Epoch 7/20
900/900 [==============================] - 4s 4ms/step - loss: 0.2305 - accuracy: 0.9176
Epoch 8/20
900/900 [==============================] - 4s 4ms/step - loss: 0.3155 - accuracy: 0.8893
Epoch 9/20
900/900 [==============================] - 4s 4ms/step - loss: 0.1835 - accuracy: 0.9367
Epoch 10/20
900/900 [==============================] - 4s 4ms/step - loss: 0.1684 - accuracy: 0.9520

In [62]:
test_loss, test_acc = model.evaluate(X_test,y_test, batch_size=1)
print(test_acc)

100/100 [==============================] - 1s 3ms/step - loss: 1.5249 - accuracy: 0.7500
0.75


In [63]:
model.save('best_model_75.h5')